## Word2Vec 하이퍼파라미터 튜닝
- vector_size 1~300까지 확인하며, 어떤 벡터 사이즈가 가장 정확한 유사도를 도출하는지 확인 시도
- 1) train, test 인물 키워드 top 20 중 BPS 5개 척도중 하나에 명확히 포함되는 키워드들을 condition으로 정의
- 2) condition들을 가장 많이 만족하는 벡터 사이즈 순위 도출

- train 데이터 : 벡터 사이즈 변화에 따른 Word2Vec 유사도

In [ ]:
def train_calculate_similarity(vector_size):
    docs = [train_top20_keywords] + [sincerity] + [excitement] + [competence] + [sophistication] + [ruggedness]
    model = Word2Vec(sentences=docs, vector_size=vector_size, window=4, hs=1, negative=0, min_count=1, sg=1)

    train_result = []
    for keyword in train_top20_keywords:
        columns = {'벡터 사이즈': vector_size, 'top20 키워드': keyword}  # 벡터 사이즈와 키워드 추가
        for category, words in [('진실성', sincerity),
                                ('흥미로움', excitement),
                                ('유능함', competence),
                                ('세련됨', sophistication),
                                ('강인함', ruggedness)]:
            sim_sum = sum(model.wv.similarity(w1=keyword, w2=word) for word in words)
            columns[category] = sim_sum / len(words)

        # 가장 유사도가 높은 단어 찾기
        max_category = max(category for category in columns.keys() if category != '벡터 사이즈' and category != 'top20 키워드' and isinstance(columns[category], (float, int)) and columns[category] == max(v for k, v in columns.items() if isinstance(v, (float))))
        columns['가장 유사한 단어'] = max_category

        train_result.append(columns)

    return pd.DataFrame(train_result)


In [ ]:
result_dfs = []
for vector_size in range(1, 300, 1):
    result_df = train_calculate_similarity(vector_size)
    result_dfs.append(result_df)

# 결과 데이터프레임들을 하나로 합치기
train_final_result = pd.concat(result_dfs, ignore_index=True)

# 결과 출력
pd.DataFrame(train_final_result)

- test 데이터 : 벡터 사이즈 변화에 따른 Word2Vec 유사도



In [ ]:
def test_calculate_similarity(vector_size):
    docs = [test_top20_keywords] + [sincerity] + [excitement] + [competence] + [sophistication] + [ruggedness]
    model = Word2Vec(sentences=docs, vector_size=vector_size, window=4, hs=1, negative=0, min_count=1, sg=1)

    test_result = []
    for keyword in test_top20_keywords:
        columns = {'벡터 사이즈': vector_size, 'top20 키워드': keyword}  # 벡터 사이즈와 키워드 추가
        for category, words in [('진실성', sincerity),
                                ('흥미로움', excitement),
                                ('유능함', competence),
                                ('세련됨', sophistication),
                                ('강인함', ruggedness)]:
            sim_sum = sum(model.wv.similarity(w1=keyword, w2=word) for word in words)
            columns[category] = sim_sum / len(words)

        # 가장 유사도가 높은 단어 찾기
        max_category = max(category for category in columns.keys() if category != '벡터 사이즈' and category != 'top20 키워드' and isinstance(columns[category], (float, int)) and columns[category] == max(v for k, v in columns.items() if isinstance(v, (float))))
        columns['가장 유사한 단어'] = max_category

        test_result.append(columns)

    return pd.DataFrame(test_result)

In [ ]:
result_dfs = []
for vector_size in range(1, 300, 1):
    result_df = test_calculate_similarity(vector_size)
    result_dfs.append(result_df)

# 결과 데이터프레임들을 하나로 합치기
test_final_result = pd.concat(result_dfs, ignore_index=True)

# 결과 출력
pd.DataFrame(test_final_result)

- train 데이터 : 특정 조건 만족 벡터사이즈 top 10 확인

In [ ]:
# 특정 조건을 만족하는 행들을 필터링하여 추출
condition1 = (train_final_result['top20 키워드'] == '솔직함') & (train_final_result['가장 유사한 단어'] == '진실성')
condition2 = (train_final_result['top20 키워드'] == '색다름') & (train_final_result['가장 유사한 단어'] == '흥미로움')
condition3 = (train_final_result['top20 키워드'] == '신선함') & (train_final_result['가장 유사한 단어'] == '흥미로움')
condition4 = (train_final_result['top20 키워드'] == '화려함') & (train_final_result['가장 유사한 단어'] == '세련됨')

filtered_df = train_final_result[condition1 | condition2 | condition3 | condition4 ]

# 카운트를 기준으로 내림차순 정렬하여 상위 10개 벡터 사이즈 추출
top_vector_sizes = vector_size_counts.sort_values(ascending=False).head(10).index

# 필터링된 결과에서 '벡터 사이즈'와 해당 카테고리의 유사도 점수 추출
train_selected_data = filtered_df[filtered_df['벡터 사이즈'].isin(top_vector_sizes)]

# 결과 출력
print(train_selected_data['벡터 사이즈'].value_counts())

- test 데이터 : 특정 조건 만족 벡터사이즈 top 10 확인

In [ ]:
condition1 = (test_final_result['top20 키워드'] == '다채함') & (test_final_result['가장 유사한 단어'] == '흥미로움')
condition2 = (test_final_result['top20 키워드'] == '독특함') & (test_final_result['가장 유사한 단어'] == '흥미로움')
condition3 = (test_final_result['top20 키워드'] == '우아함') & (test_final_result['가장 유사한 단어'] == '세련됨')
condition4 = (test_final_result['top20 키워드'] == '강함') & (test_final_result['가장 유사한 단어'] == '강인함')

filtered_df = final_result[condition1 | condition2 | condition3 | condition4 ]

# 필터링된 결과에서 '벡터 사이즈' 별로 카운트 계산
vector_size_counts = filtered_df['벡터 사이즈'].value_counts()

# 카운트를 기준으로 내림차순 정렬하여 상위 10개 벡터 사이즈 추출
top_vector_sizes = vector_size_counts.sort_values(ascending=False).head(10).index

# 상위 벡터 사이즈에 해당하는 데이터 추출
test_selected_data = filtered_df[filtered_df['벡터 사이즈'].isin(top_vector_sizes)]

# 결과 출력
print(test_selected_data['벡터 사이즈'].value_counts())